# 딥러닝
**딥러닝(deep learning)** 은 (한 개 이상의 은닉층을 지닌) '깊은' 신경망을 의미한다. 

요즘에는 다양한 신경망 구조를 모두 아우르는 용어가 되었다.

## 19.1 텐서
1차원 배열은 벡터, 2차원 배열은 행렬으로 명명하였다. 복잡한 신경망 구조에서는 더 고차원 배열이 필요한데 대부분의 신경망 라이브러리의 명칭을 따라서 n차원 배열을 **텐서(tensor)**라고 하겠다.

텐서를 리스트로 정의해보자

In [1]:
Tensor = list

아래 코드는 텐서의 크기를 반환해 주는 도우미 함수(helper function)

In [2]:
from typing import List

def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes

In [3]:
assert shape([1, 2, 3]) == [3]
assert shape([[1, 2], [3, 4], [5, 6]]) == [3, 2]

텐서마다 차원의 수가 다양하기  때문에 재귀적으로 텐서를 살펴봐야 한다. 

밑의 함수는 1차원 벡터부터 재귀적으로 고차원의 텐서까지 살펴볼 것이다.

In [4]:
def is_1d(tensor: Tensor) -> bool:
    """
    만약 tensor[0]이 리스트라면 고차원 텐서를 의미
    그러지 않으면 1차원 벡터를 의미
    """
    return not isinstance(tensor[0], list)

In [6]:
assert is_1d([1, 2, 3])
assert not is_1d([[1, 2], [3, 4]])

이런 함수를 사용해 텐서 안의 모든 값의 합을 반환해주는 tensor_sum 함수를 생성할 수 있다.

In [7]:
def tensor_sum(tensor: Tensor) -> float:
    """텐서 안의 모든 값의 합을 반환"""
    if is_1d(tensor):
        return sum(tensor)  # 벡터의 경우, 파이썬 기본 함수인 sum을 사용
    else:
        return sum(tensor_sum(tensor_i)  # 벡터가 아닌 경우, 각 행별 tensor_sum을 호출하고
                  for tensor_i in tensor)  # 결과값을 더해준다.

In [9]:
assert tensor_sum([1,2,3]) == 6
assert tensor_sum([[1,2], [3,4]]) == 10

위 else statement에 tensor_sum이 재귀적으로 사용되었다. 

매번 이렇게 재귀적으로 구현하는 것은 번거로우니 이러한 기능을 대신해 주는 도우미 함수를 만들자. 먼저 텐서 안의 모든 값에 일괄적으로 함수를 적용할 수 있게 해주는 함수를 만들어보자.

In [14]:
from typing import Callable

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    """텐서 안의 모든 값에 f를 적용"""
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

In [15]:
assert tensor_apply(lambda x: x + 1, [1, 2, 3]) == [2, 3, 4]
assert tensor_apply(lambda x: 2 * x, [[1, 2], [3, 4]]) == [[2, 4], [6, 8]]

위 도우미 함수를 사용하면 주어진 크기와 동일한 0 텐서를 만들 수 있다.

In [17]:
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0, tensor)

In [18]:
assert zeros_like([1,2,3]) == [0, 0, 0]
assert zeros_like([[1,2], [3, 4]]) == [[0, 0], [0, 0]]

그리고 두 텐서의 대칭되는 값에 일괄적으로 함수를 적용할 수 있게 해주는 함수를 만들어 보자.

In [19]:
def tensor_combine(f: Callable[[float, float], float],
                  t1: Tensor,
                  t2: Tensor) -> Tensor:
    """두 텐서의 대칭되는 t1과 t2에 일괄적으로 함수를 적용"""
    if is_1d(t1):
        return [f(x, y) for x, y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i)
               for t1_i, t2_i in zip(t1, t2)]

In [20]:
import operator
assert tensor_combine(operator.add, [1,2,3], [4,5,6]) == [5,7,9]
assert tensor_combine(operator.mul, [1,2,3], [4,5,6]) == [4,10,18]

## 19.2 층 추상화
18장에서는 각각 sigmoid(dot(weights, inputs))를 계산해 주는 두 층으로 구성된 단순한 신경망을 만들어 보았다.

이러한 구조는 실제 뉴런이 작동하는 방식과 비슷하지만, 딥러닝에서는 다양한 신경망 구조를 사용할 것이다. 가령 각 뉴런이 이전 입력값을 기억하도록 만들거나 시그모이드 대신 다른 활성화 함수(activation function)를 사용할 것이다. 또한 두 층보다 더 깊은 신경망을 만들 것이다. (feed_forward 함수로 두 층 이상의 신경망을 만드는 것은 구현했지만 그래디언트를 계산하는 것은 아직 구현하지 않았다.)

이번 장에서는 이러한 다양한 신경망을 구현해 볼 것이다. 가장 핵심적인 **추상화** 는 신경망의 각 층을 나타내는 Layer이다. Layer에서는 입력값에 특정 함수를 적용하거나 역전파를 할 수 있어야 한다. 18장에서 구현한 신경망은 '선형'층 위에 '시그모이드' 층으로 구성된 신경망이라고 볼 수 있다.

새로운 용어라 어색할 수도 있지만 이렇게 받아들이는 것이 다양한 신경망 구조를 이해하는 데 도움이 될 것이다.

In [21]:
from typing import Iterable, Tuple

class Layer:
    """
    딥러닝 신경망은 Layer들로 구성되어 있다.
    각 Layer별로 순방향으로 입력값에 어떤 계산을 하고
    역방향으로 그래디언트를 전파해야 한다.
    """
    def forward(self, input):
        """
        타입이 명시되어 있지 않은 것을 유의하자.
        입력층과 출력값의 타입의 제한하지 않을 것이다.
        """
        raise NotImplementedError
            
    def backward(self, gradient):
        """
        역방향에서도 그래디언트의 타입을 제한하지 않을 것이다.
        메서드를 호출할 때 유의하자.
        """
        raise NotImplementedError
    
    def params(self) -> Iterable[Tensor]:
        """
        해당 층의 파라미터를 반환
        기본적으로 아무것도 반환하지 않을 것이다.
        만약 특정 층에서 반환할 파라미터가 없다면 구현할 필요가 없다.
        """
        return ()
    
    def grads(self) -> Iterable[Tensor]:
        """
        params()처럼 그래디언트를 반환
        """
        return ()

forward, backward 메서드는 곧 구현할 것이다. 그리고 경사 하강법으로 신경망의 파라미터를 학습하기 위해 각 층의 파라미터와 그래디언트를 반환해 줄 수 있어야 한다. 

시그모이드 층과 비슷한 층에서는 파라미터를 업데이트할 필요가 없기 때문에 이러한 경우를 위해 기본값 설정 또한 해주었다.

먼저 시그모이드 층을 구현해보자.

In [22]:
import math

def sigmoid(t: float) -> float:
    return 1 / (1 + math.exp(-t))

In [23]:
class Sigmoid(Layer):
    def forward(self, input: Tensor) -> Tensor:
        """
        입력된 Tensor의 모든 값에 sigmoid를 계산
        backpropagation을 위해 결괏값을 저장
        """
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids
    
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad, self.sigmoids, gradient)

몇 가지 유의할 점이 있다. 역전파를 위해 순방향에서 계산되는 시그모이드 값을 모두 저장할 것이다. 앞으로 층의 순방향에서 계산되는 값은 대부분 저장할 것이다. 

두 번째로 sig * (1 - sig) * grad가 어디서 나왔는지 궁금할 것이다. 이 값은 18장에서 다룬 미적분과 연쇄법칙으로 계산된 output * (1 - output) * (output - target)이다.]

마지막으로 tensor_apply와 tensor_combine 함수를 사용했다. 앞으로 구현할 모든 층에서는 기본적으로 두 함수를 사용할 것이다.

## 19.3 선형 층
18장에서 구현한 신경망을 기반으로 뉴런의 dot(weights, inputs)를 나타내는 선형 층(linear layer)을 구현할 수 있다.

선형 층의 초깃값은 임의로 생성할 것이다.

사실 임의로 파라미터의 초깃값을 설정하는 것은 매우 중요하다. 파라미터의 초깃값에 따라 신경망의 학습 속도가 (혹은 학습 여부 자체가) 결정된다. 만약 초깃값이 너무 크면 활성화 함수의 그래디언트는 0에 가깝게 된다. 그래디언트가 0에 가까운 파라미터는 경사 하강법으로 학습을 할 수 없게 된다.

따라서 파라미터를 임의로 생성해 주는 세 가지 방법을 구현할 것이다. 먼저 random.random()으로 균등 분포를 구현하여, 파라미터의 초깃값을 0과 1 사이 임의의 값으로 설정할 것이다. 두 번째 방법으로 표준정규분표에서 임의의 초깃값을 생성할 것이다. 마지막으로 딥러닝에서 자주 쓰이는 Xavier 초기화에서는 평균이 0이고 편차가 2 / (입력 값의 개수 + 출력 값의 개수)인 정규분포에서 임의의 초깃값을 생성할 것이다. 세 가지 방법 모두 random_uniform과 random_normal 함수로 구현할 것이다. 

In [29]:
import random

def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p: float,
                      mu: float = 0,
                      sigma: float = 1,
                      tolerance: float = 0.00001) -> float:
    """이진 검색을 사용해 역함수를 근사"""
    # 표준정규분포가 아니라면 표준정규분포로 변환
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z = -10.0  # normal_cdf(-10)은 0에 근접
    hi_z = 10.0  # normal_cdf(10)은 1에 근접
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2  # 중간 값
        mid_p = normal_cdf(mid_z)  # 중간 값의 누적분포 값을 계산
        if mid_p < p:
            low_z = mid_z  # 중간 값이 너무 작다면 더 큰 값들을 검색
        else:
            hi_z = mid_z  # 중간 값이 너무 크다면 더 작은 값들을 검색
    
    return mid_z

In [30]:
def random_uniform(*dims: int) -> Tensor:
    if len(dims) == 1:
        return [random.random() for _ in range(dims[0])]
    else:
        return [random_uniform(*dims[1:]) for _ in range(dims[0])]

In [31]:
def random_normal(*dims: int, 
                 mean: float = 0.0,
                 variance: float = 1.0) -> Tensor:
    if len(dims) == 1:
        return [mean + variance * inverse_normal_cdf(random.random())
               for _ in range(dims[0])]
    else:
        return [random_normal(*dims[1:], mean=mean, variance=variance)
               for _ in range(dims[0])]

In [32]:
assert shape(random_uniform(2, 3, 4)) == [2, 3, 4]
assert shape(random_normal(5, 6, mean=10)) == [5, 6]

그리고 random_tensor 함수로 감싸주자.

In [33]:
def random_tensor(*dims: int, init: str = 'normal') -> Tensor:
    if init == 'normal':
        return random_normal(*dims)
    elif init == 'uniform':
        return random_uniform(*dims)
    elif init == 'xavier':
        variance = len(dims) / sum(dims)
        return random_normal(*dims, variance=variance)
    else:
        raise ValueError(f"unkown init: {init}")

이제 선형 층을 구현할 준비가 끝났다. 먼저 입력값의 차원 수(각 뉴런별 파라미터 개수), 출력값의 차원 수(뉴런의 개수), 초기화 방법을 명시해 줘야 한다. 

In [35]:
Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

In [36]:
class Linear(Layer):
    def __init__(self,
                input_dim: int,
                output_dim: int,
                init: str = 'xavier') -> None:
        """
        output_dim개의 뉴런과 각 뉴런별 input_dim개의 파라미터로 (편향 제외)
        구성된 층
        """
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # self.w[o]는 o번째 뉴런의 파라미터
        self.w = random_tensor(output_dim, input_dim, init=init)
        
        # self.b[o]는 o번째 뉴런의 편향
        self.b = random_tensor(output_dim, init=init)

선형 층의 forward 메서드는 쉽게 구현할 수 있다. 뉴런별 한 개의 값을 벡터로 저장할 것이다. 각 뉴런의 입력값과 파라미터를 곱하고 편향을 더해주면 출력값이 계산된다.

In [37]:
def forward(self, input: Tensor) -> Tensor:
    # 역방향을 위해 input을 저장
    self.input = input
    
    # 뉴런의 결괏값을 벡터로 반환
    return [dot(input, self.w[o]) + self.b[o]
           for o in range(self.output_dim)]

backward 메서드는 조금 더 복잡하지만, 미적분을 알고 있다면 금방 이해할 수 있다.

In [39]:
def backward(self, gradient: Tensor) -> Tensor:
    # 각 b[o]는 output[o]에 더해진다.
    # 즉, b의 그래디언트는 output의 그래디언트과 동일하다는 것을 의미
    self.b_grad = gradient
    
    # 각 w[o][i]를 input[i]에 곱하고 output[o]에다 더해 준다.
    # 즉, 그래디언트는 input[i] * gradient[o]
    self.w_grad = [[self.input[i] * gradient[o]
                   for i in range(self.input_dim)]
                   for o in range(self.output_dim)]
    # input[i]에 각 w[o][i]를 곱하고
    # output[o]에 더해 주기 때문에 그래디언트는 w[o][i] * gradient[o]를
    # 모두 더해 준 값
    return [sum(self.w[o][i] * gradien[o] for o in range(self.output_dim))
           for i in range(self.input_dim)]

마지막으로 params와 grads 메서드를 만들어 보자. 선형 층에는 두 개의 파라미터와 그래디언트가 있다는 것을 기억하자.

In [40]:
def params(self) -> Iterable[Tensor]:
    return [self.w, self.b]

def grads(self) -> Iterable[Tensor]:
    return [self.w_grad, self.b_grad]

## 19.4 순차적인 층으로 구성된 신경망
신경망을 순차적인 층으로 구성되어 있다고 생각해 볼 수 있다. 그렇다면 여러층을 하나의 층으로 표현할 수도 있을 것이다. 즉, 하나의 신경망 자체를 Layer의 메서드를 활용해서 하나의 층으로 표현해 보자.

In [42]:
from typing import List

class Sequential(Layer):
    """
    하나의 Layer에는 실제 여러 층이 포함되어 있다. 
    각 층의 출력값이 
    다음 층의 입력값이 된다는 것을 꼭 이해하고 넘어가자
    """
    
    def __init__(self, layers: List[Layer]) -> None:
        self.layers = layers
        
    def forward(self, input):
        """순차적으로 각 층의 입력값을 전파"""
        for layer in self.layers:
            input = layer.forward(input)
        return input
    
    def backward(self, gradient):
        """역방향으로 각 층의 그래디언트를 전파"""
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)
        return gradient
    
    def params(self) -> Iterable[Tensor]:
        """각 층별 파라미터를 반환"""
        return (param for layer in self.layers for param in layer.params())
    
    def grads(self) -> Iterable[Tensor]:
        """각 층별 그래디언트를 반환"""
        return (grad for layer in self.layers for grad in layer.grads())

18장에서 XOR 게이트를 위해 만들었던 신경망을 다음과 같이 만들 수 있다.

In [43]:
xor_net = Sequential([
    Linear(input_dim=2, output_dim=2),
    Sigmoid(),
    Linear(input_dim=2, output_dim=1),
    Sigmoid()
])

## 19.5 손실 함수와 최적화
지금까지는 손실함수(loss function)와 그래디언트 함수를 직접 명시하였다. 이번에는 다양한 손실 함수를 살펴볼 것이며, 손실 함수와 그래디언트 계산을 loss라는 클래스로 추상화할 것이다.

In [ ]:
class Loss:
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        """예측값이 얼마나 정확한가?"""